In [ ]:
import requests
import pandas
import os 

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [5]:
url = f"http://api.citybik.es/v2/networks/mobibikes"

response = requests.get(url)
if response.status_code == 200:
    cb_data = response.json()
    cb_network = cb_data.get('network',{})
    stations = cb_network.get('stations',[])

    for station in stations:
        name = station.get('name')
        latitude = station.get('latitude')
        longitude = station.get('longitude')
        bikes_available = station.get('free_bikes')
else: 
    print("request returned negative")

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [ ]:
citybikes_url = "http://api.citybik.es/v2/networks/mobibikes"
fsq_api_key=os.getenv("FSQ_API_KEY")

poi_data = []

response = requests.get(citybikes_url)
if response.status_code == 200:
    cb_data = response.json()
    cb_network = cb_data.get('network',{})
    stations = cb_network.get('stations',[])

    for station in stations:
        name = station.get('name')
        latitude = station.get('latitude')
        longitude = station.get('longitude')
        bikes_available = station.get('free_bikes')

        print(f"Station: {name}, Coordinates: ({latitude},{longitude}), Free Bikes: {bikes_available}")

        fsq_url = "https://api.foursquare.com/v3/places/search"

        fsq_params = {
            "ll": f"{latitude},{longitude}",  # Correct parameter for latitude and longitude
            "radius": 1000,
            "categories": "19045,13035",  # Coffee shops
            "limit": 50,
            "fields": "name,location,geocodes,rating" #geocodes = latitude,longitude
        }

        fsq_headers = {
            "accept": "application/json",
            "Authorization": fsq_api_key
        }

        fsq_response = requests.get(fsq_url, params=fsq_params, headers=fsq_headers)
        if fsq_response.status_code == 200:
            fsq_data = fsq_response.json()

            print(f"\nPlaces near {name}:\n")
            for venue in fsq_data.get('results', []):
                venue_name = venue.get('name')
                venue_address = venue.get('location', {}).get('formatted_address', 'No address provided')
                venue_longitude = venue.get('geocodes', {}).get('main', {}).get('longitude')
                venue_latitude = venue.get('geocodes', {}).get('main', {}).get('latitude')
                venue_rating = venue.get('rating', 'No rating provided')

                print(f"Name: {venue_name}, Address: {venue_address}, Coordinates: ({venue_latitude}, {venue_longitude}), Rating: {venue_rating}")
            print("\n")
        else:
            print(f"Foursquare request failed with status code {fsq_response.status_code}") ## was having errors around here so I wrote this part to get the status code so i can identify the issues.
else:
    print(f"CityBikes request failed with status code {response.status_code}")

df = pandas.DataFrame(poi_data)
df.to_csv('foursquare_data.csv', index=False)
print(df)

Station: Chilco & Barclay, Coordinates: (49.291909,-123.140713), Free Bikes: 6

Places near Chilco & Barclay:

Name: Cardero Bottega, Address: 1016 Cardero St (at Comox St.), Vancouver BC V6G 2H1, Coordinates: (49.287092, -123.135598), Rating: 8.7
Name: Pappa Roti, Address: 1505 Robson St (at Nicola St), Vancouver BC V6G 1C3, Coordinates: (49.28892, -123.13113), Rating: 8.7
Name: Delany's Coffee House, Address: 1105 Denman St (at Comox St.), Vancouver BC V6G 2M7, Coordinates: (49.288307, -123.140488), Rating: 7.9
Name: Greenhorn Cafe, Address: 994 Nicola St (at Nelson St), Vancouver BC V6G 2C8, Coordinates: (49.286624, -123.133935), Rating: 8.5
Name: JJ Bean Coffee Roasters, Address: 1209 Bidwell St (Davie), Vancouver BC V6G 2K7, Coordinates: (49.286217, -123.140552), Rating: 8.0
Name: Blenz Coffee, Address: 935 Denman St (Barclay St.), Vancouver BC V6G 2L9, Coordinates: (49.290242, -123.137571), Rating: 7.5
Name: Starbucks, Address: 1795 Davie St (at Denman St), Vancouver BC V6G 1W5, 

KeyboardInterrupt: 

Put your parsed results into a DataFrame

In [ ]:
citybikes_url = "http://api.citybik.es/v2/networks/mobibikes"

poi_data = []

response = requests.get(citybikes_url)
if response.status_code == 200:
    cb_data = response.json()
    cb_network = cb_data.get('network',{})
    stations = cb_network.get('stations',[])

    for station in stations:
        name = station.get('name')
        latitude = station.get('latitude')
        longitude = station.get('longitude')
        bikes_available = station.get('free_bikes')

        print(f"Station: {name}, Coordinates: ({latitude},{longitude}), Free Bikes: {bikes_available}")

        fsq_url = "https://api.foursquare.com/v3/places/search"

        fsq_params = {
            "ll": f"{latitude},{longitude}",  
            "radius": 1000,
            "categories": "19045,13035",  # Coffee shops
            "limit": 50,
            "fields": "name,location,geocodes,rating"
        }

        fsq_headers = {
            "accept": "application/json",
            "Authorization": fsq_api_key
        }

        fsq_response = requests.get(fsq_url, params=fsq_params, headers=fsq_headers)
        if fsq_response.status_code == 200:
            fsq_data = fsq_response.json()

            for poi in fsq_data.get('results', []):
                poi_name = poi.get('name')
                poi_address = poi.get('location', {}).get('formatted_address', 'No address provided')
                poi_longitude = poi.get('geocodes', {}).get('main', {}).get('longitude')
                poi_latitude = poi.get('geocodes', {}).get('main', {}).get('latitude')
                poi_rating = poi.get('rating', 'No rating provided')


            poi_data.append({
                'Station Name': name,
                'Station Coordinates': (latitude, longitude),
                'Bikes Available': bikes_available,
                'Name': poi_name,
                'Address': poi_address,
                'Coordinates': (poi_latitude,poi_longitude),
                'Rating': poi_rating
            })

        else:
            print(f"Foursquare request failed: {fsq_response.status_code}")

else:
    print(f"CityBikes request failed: {response.status_code}")

df = pandas.DataFrame(poi_data)
df.to_csv('foursquare_data.csv', index=False)
print(df)



Station: Chilco & Barclay, Coordinates: (49.291909,-123.140713), Free Bikes: 9
Station: St George & Broadway, Coordinates: (49.262321,-123.09306), Free Bikes: 4
Station: Britannia Parking Lot, Coordinates: (49.275882,-123.071865), Free Bikes: 11
Station: Morton & Denman, Coordinates: (49.28803,-123.142135), Free Bikes: 6
Station: Thornton & National, Coordinates: (49.273777,-123.092723), Free Bikes: 6
Station: Burrard & 14th, Coordinates: (49.259469,-123.145718), Free Bikes: 4
Station: Olympic Village Station, Coordinates: (49.266314,-123.116011), Free Bikes: 4
Station: 8th & Macdonald, Coordinates: (49.264928,-123.168068), Free Bikes: 11
Station: 30th & Ontario, Coordinates: (49.243375,-123.104978), Free Bikes: 7
Station: Lakewood & Triumph, Coordinates: (49.283968,-123.061615), Free Bikes: 7
Station: 13th & Cambie, Coordinates: (49.259579,-123.114769), Free Bikes: 8
Station: Arbutus & Nanton, Coordinates: (49.247854,-123.153422), Free Bikes: 10
Station: 14th & Fraser, Coordinates: (4

KeyboardInterrupt: 

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [6]:
url = f"http://api.citybik.es/v2/networks/mobibikes"

response = requests.get(url)
if response.status_code == 200:
    cb_data = response.json()
    cb_network = cb_data.get('network',{})
    stations = cb_network.get('stations',[])

    for station in stations:
        name = station.get('name')
        latitude = station.get('latitude')
        longitude = station.get('longitude')
        bikes_available = station.get('free_bikes')
else: 
    print("request returned negative")

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [ ]:
import pandas as pd
import requests


cb_url = "http://api.citybik.es/v2/networks/mobibikes"
yelp_api_key = os.getenv("YELP_API_KEY")
yelp_url = "https://api.yelp.com/v3/businesses/search"                  #defining the urls and api key needed.

response = requests.get(cb_url)
if response.status_code == 200:                                         # checking response status to see if I get a positive response
    cb_data = response.json()
    cb_network = cb_data.get('network', {})
    stations = cb_network.get('stations', [])

    all_results = []  

    for station in stations:
        name = station.get('name')
        latitude = station.get('latitude')
        longitude = station.get('longitude')
        bikes_available = station.get('free_bikes')                    # loops through citybikes to get each station and their coordinates so we can look for POI in the yelp API

        results = []  
        offset = 0
        limit = 50  

        while True:
            headers = {
                'Authorization': yelp_api_key  
            }
            params = {
                'term': 'coffee',                                      # Parameters I'm looking to get in response/filter off of in Yelp.
                'latitude': latitude,
                'longitude': longitude,
                'radius': 1000,  
                'limit': limit,
                'offset': offset
            }

            yelp_response = requests.get(yelp_url, headers=headers, params=params)

            if yelp_response.status_code == 200:
                yelp_data = yelp_response.json()
                businesses = yelp_data.get('businesses', [])

                if not businesses:  
                    break                                               # checks if anything is returned. If the list is empty, it ends the loop to avoid over-usage of requests to the API

                for business in businesses:
                    results.append({
                        'Station Name': name,
                        'Bikes Available': bikes_available,
                        'Business Name': business['name'],
                        'Business Rating': business['rating'],
                        'Latitude': business['coordinates']['latitude'],
                        'Longitude': business['coordinates']['longitude']           # Collects relative information for each business.
                    })

                offset += limit
   
            else:
                print(f"Yelp API request failed for station {name} with status code: {yelp_response.status_code}")          # was getting alot of errors in the beginning so i had the status_code returned so I can troubleshoot.
                break

        all_results.extend(results)

    df = pd.DataFrame(all_results)
    df.to_csv('yelp_data.csv', index=False)                                                                                 #saving database to later usage.


Yelp API request failed for station Chilco & Barclay with status code: 429
Yelp API request failed for station St George & Broadway with status code: 429
Yelp API request failed for station Britannia Parking Lot with status code: 429
Yelp API request failed for station Morton & Denman with status code: 429
Yelp API request failed for station Thornton & National with status code: 429
Yelp API request failed for station Burrard & 14th with status code: 429
Yelp API request failed for station Olympic Village Station with status code: 429
Yelp API request failed for station 8th & Macdonald with status code: 429
Yelp API request failed for station 30th & Ontario with status code: 429
Yelp API request failed for station Lakewood & Triumph with status code: 429
Yelp API request failed for station 13th & Cambie with status code: 429
Yelp API request failed for station Arbutus & Nanton with status code: 429
Yelp API request failed for station 14th & Fraser with status code: 429
Yelp API request

KeyboardInterrupt: 

Put your parsed results into a DataFrame

In [1]:
##done in previous cellblock 

Total number of businesses collected: 72


# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

From a general overview from the cell output blocks, The Yelp API provided me with more data. This is due to Yelp being a more familiar online guide to public businesses. Therefore, having more up to date data inputted by public users. I can also confirm that Yelp has more user reviews and photos; along with general information of businesses such as business hours, services, events, and so on.

Get the top 10 restaurants according to their rating

dssd

In [ ]:
import pandas as pd

foursquare_df = pd.read_csv('foursquare_data.csv')                                          # opening the csv file 
foursquare_df['Rating'] = foursquare_df['Rating'].replace('No rating provided',pd.NA)       # replaces anything that has 'No ratings provided' with an NA value, turning it into an integar instead of a string
foursquare_df.dropna(subset=['Rating'], inplace=True)                                       # Droping all said NA values so we are only left with businesses with ratings.
top_resturants = foursquare_df.sort_values(by='Rating',ascending=False).head(10)            # sort values by descending, limiting to only 10.

print(top_resturants[['Name','Rating']])

                   Name Rating
241        Blenz Coffee    6.9
11            Starbucks    6.8
104        Blenz Coffee    6.0
157  Waves Coffee House    5.7
250        Blenz Coffee    5.6
231        Blenz Coffee    5.6
242      Blue Edge Cafe    5.5
234      Blue Edge Cafe    5.5
206         Little Cafe    5.5
0        Blue Edge Cafe    5.5
